# Resting state fMRI preprocessing
This notebook contains preprocessing tailored precision mapping in children. 

### 1. General fMRI processing
* Slice-time correction
* Rigid realignment (also extract DVARS, FD, motion params)
* Co-registration to the sMRI (T1-weighted structural MRI resampled to 2x2x2mm voxels)

### 2. Resting state processing
* Derive temporal mask based on FD and DVARS
* Demean and detrend
* De-noise to remove:
    - Mean timeseries for that voxel
    - Component noise associated with white matter and CSF- delete the GM and smooth what is left
    - Component noise associated with background signal - delete brain and smooth what's left
    - the averaged timeseries
    - motion regressors
    - Motion derivatives (lagged 8 times)
    - Squared derivatives (lagged 8 times)
* Interpolate timepoints using spectral based interpolation
* Bandpass filter
* delete interpolated timepoints and concatenate runs

### 3. Cortical registration and CIFTI creation
* Register fMRI to cortical surfaces
* Make CIFTI file that includes cortical, subcortical, and cerebellar data

In [4]:
#import packages
from os import listdir, makedirs
from os.path import isdir
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber # Data i/o
from nipype.interfaces.utility import IdentityInterface, Function     # utility
from nipype.pipeline.engine import Node, Workflow, MapNode, JoinNode        # pypeline engine
from nipype.interfaces.nipy.preprocess import Trim
from nipype.interfaces.ants import N4BiasFieldCorrection
from nipype.interfaces.fsl import SliceTimer, MCFLIRT, FLIRT, BET, Merge
from nipype.interfaces.fsl.utils import Reorient2Std, MotionOutliers
from nipype.interfaces.fsl.maths import ApplyMask, MeanImage
from nipype.interfaces.freesurfer import Resample, Binarize, BBRegister
from nipype.algorithms.confounds import CompCor
from nipype.interfaces.afni.preprocess import Bandpass
from nipype.interfaces.afni.utils import AFNItoNIFTI
from pandas import DataFrame, read_csv

#set output file type for FSL to NIFTI_GZ
from nipype.interfaces.fsl.preprocess import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# Set study variables
study_home = '/moochie/user_data/CamachoCat/5YOP'
raw_data =  '/moochie/study_data/5YOP/MRI_processing'
output_dir = study_home + '/proc/preprocessing'
workflow_dir = study_home + '/workflows'
custom_timings = study_home + '/misc/slice_timing.txt'
fs_dir = study_home + '/freesurfer'
session_info = read_csv(study_home + '/misc/session_info.csv',index_col=None)

subject_ids = session_info['subject_id'].unique().tolist()
sessions_lists = []
for sub in subject_ids:
    sessions_lists.append(session_info[session_info['subject_id']==sub]['run_name'].tolist())

proc_cores = 6 # number of cores of processing for the workflows

interleave = True
TR = 1.1 # in seconds
slice_dir = 3 # 1=x, 2=y, 3=z
resampled_voxel_size = (2,2,2)

highpass_freq = 0.008 #in Hz
lowpass_freq = 0.09 #in Hz

In [5]:
## File handling Nodes

# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subject_id','sessions']),
                     name='infosource')
infosource.iterables = [('subject_id', subject_ids),('sessions', sessions_lists)]

# Datasink- where our select outputs will go
substitutions = [('_subject_id_', '')]
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir
datasink.inputs.substitutions = substitutions

## Process T1 anat for fMRI processing
1. Resample skullstripped T1w anat
2. Resample segmentation 
3. Create tissue masks:
    - GM only
    - Non-GM only
    - Full brain (no ventricles)
    - Full brain (with ventricles and dilated)
    - Extra-cerebral space
4. Resample surfaces

In [ ]:
def make_masks_freesurfer(aseg_nifti):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nibabel import load, save, Nifti1Image
    from numpy import zeros
    from nipype.interfaces.freesurfer import Binarize
    from os.path import abspath
    
    # load the subject's segmentation volumes
    aseg_img = load(aseg_nifti)
    aseg_data = aseg_img.get_data()
    
    # list indices for each tissue type (from freesurfer aseg)
    wm = [2, 7, 41, 46, 60, 16, 77, 251, 252, 253, 254, 255]
    gm = [3, 6, 8, 9, 10, 11, 12, 13, 17, 18, 26, 42, 47, 49, 50, 51, 52, 53, 54, 58]
    csf = [4, 5, 14, 15, 24, 63]
    non_gm = wm + csf
    whole_brain = wm_ids + gm_ids + ventricles_ids
    
    # preallocate zeros for the tissue masks
    wm_combined = zeros(aseg_data.shape)
    gm_combined = zeros(aseg_data.shape)
    csf_combined = zeros(aseg_data.shape)
    non_gm_combined = zeros(aseg_data.shape)
    whole_brain_combined = zeros(aseg_data.shape)
    
    # put labels into lists to make looping easier
    tissues_idx = [wm, gm, csf, non_gm, whole_brain]
    tissues_data = [wm_combined, gm_combined, csf_combined, non_gm_combined, whole_brain_combined]
    tissue_name = ['wm','gm','csf','non_gm','whole_brain']
    
    # initialize the binarize class
    binarize = Binarize()
    binarize.inputs.dilate = 1
    binarize.inputs.erode = 1
    binarize.inputs.min = 0.5
    
    # isolate labels for specific regions
    for x in range(0,4):
        for label in tissues_idx[x]:
            tissues_data[x][aseg_data==label] = 1
        tmp_img = Nifti1Image(tissues_data[x],header=aseg_img.header, affine=aseg_img.affine)
        save(tmp_img, tissue_name[x] + '_mask.nii.gz')
        
        binarize.inputs.in_file = tissue_name[x] + '_mask.nii.gz'
        binarize.inputs.binary_file = tissue_name[x] + '_mask.nii.gz'
        binarize.run()
    
    # make dilated brain mask
    binarize.inputs.in_file = 'whole_brain_mask.nii.gz'
    binarize.inputs.binary_file = 'whole_brain_D1_mask.nii.gz'
    binarize.inputs.dilate = 1
    binarize.inputs.erode = None
    binarize.run()
    
    # gather up the mask filepaths to return
    wm_csf_mask = abspath('non_gm_mask.nii.gz')
    brain_mask = abspath('whole_brain_mask.nii.gz')
    gm_only_mask = abspath('gm_mask.nii.gz')
    brain_mask_dilated = abspath('whole_brain_D1_mask.nii.gz')
    
    return(wm_csf_mask, brain_mask, gm_only_mask, brain_mask_dilated)

def invert_masks(mask_file):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nibabel import load, save, Nifti1Image
    from os.path import abspath, basename
    from numpy import zeros
    
    # load mask data
    mask_img = load(mask_file)
    mask_data = mask_img.get_data()
    
    # preallocate new mask data
    inv_data = zeros(mask_data.shape)
    
    # invert the 1s and 0s
    inv_data[mask_data==0] = 1
    
    #save as a new file
    inv_img = Nifti1Image(inv_data, header = mask_data.header, affine = mask_img.affine)
    save(inv_img, 'inverted_' + basename(maskfile))
    inverted_mask = abspath('inverted_' + basename(maskfile))
    
    return(inverted_mask)

In [ ]:
# FreeSurferSource node to grab processed T1w data from freesurfer
t1w_source = Node(FreeSurferSource(subject_dir=fs_dir), name='t1w_source')

# resample brain file
resample_anat = Node(MRIConvert(vox_size=resampled_voxel_size, 
                                out_type='niigz', out_file='resampled_anat.nii.gz'), name='resample_anat')

# resample segmentation file
resample_seg = Node(MRIConvert(vox_size=resampled_voxel_size, 
                               resample_type='nearest', 
                               out_type='niigz'), name='resample_seg')

# make masks
make_masks = Node(Function(input_names=['aseg_nifti'], 
                           output_names=['wm_csf_mask', 'brain_mask', 'gm_only_mask', 'brain_mask_dilated'], 
                           function=make_masks_freesurfer), name='make_masks')

#invert the brain mask
invert_mask = Node(Function(input_names=['mask_file'], 
                            output_names=['inverted_mask'], 
                            function=invert_masks), name='invert_mask')

In [ ]:
anatpreproc = Workflow(name='fmri_anat_preprocflow')
anatpreproc.connect([(infosource, t1w_source, [('subject_id','subject_id')]),
                     (t1w_source, resample_anat, [('brain','in_file')]),
                     (t1w_source, resample_seg, [('aseg','in_file')]),
                     (resample_seg, make_masks, [('out_file','aseg_nifti')]),
                     (make_masks, invert_mask, [('brain_mask','mask_file')]),
                     
                     (make_masks, datasink, [('wm_csf_mask','wm_csf_mask'),
                                             ('brain_mask','brain_mask'),
                                             ('gm_only_mask','gm_mask'),
                                             ('brain_mask_dilated','brain_mask_dilated')]),
                     (resample_anat, datasink, [('out_file','resampled_t1w_anat')]),
                     (invert_mask, datasink, [('inverted_mask','background_mask')])
                    ])
anatpreproc.base_dir = workflow_dir
anatpreproc.write_graph(graph2use='flat')
anatpreproc.run('MultiProc', plugin_args={'n_procs': proc_cores})

## Preprocess fMRI resting state data
These nodes and workflow (preprocflow) perform basic preprocessing to align the functional volumes into a common space.
1. Reorient images to standard space
2. Reslice the structural image to 2mm isotropic
3. Functional image slice time correction
4. Rigid realignment to first volume of functional image
5. Coregistration of functional images to structural image

In [3]:
## File handling Nodes
anat_template = {'brain_mask': output_dir + '/brain_mask/{subject_id}/whole_brain_mask.nii.gz',
                 'resliced_anat': output_dir + '/resampled_t1w_anat/{subject_id}/resampled_anat.nii.gz'}
selectanat = Node(SelectFiles(anat_template), name='selectanat')

# Data grabber- select fMRI
func_template = {'func':raw_data + '/{subject_id}/FUNC_NIFTIS/{sessions}.nii.gz'}
selectfunc = MapNode(SelectFiles(func_template), name='selectfunc',iterfield=['sessions'])

In [ ]:
# Data QC nodes
def create_coreg_plot(epi,anat):
    import os
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nilearn import plotting
    from nipype.interfaces.nipy.preprocess import Trim
    from os.path import abspath
    
    tr = Trim()
    tr.inputs.in_file = epi[0]
    tr.inputs.end_index = 1
    tr.inputs.out_file = 'firstvol.nii.gz'
    tr.run()
    
    epi = abspath('firstvol.nii.gz')
    
    coreg_filename='coregistration.png'
    display = plotting.plot_anat(epi, display_mode='ortho',
                                 draw_cross=False,
                                 title = 'coregistration to anatomy')
    display.add_edges(anat)
    display.savefig(coreg_filename) 
    display.close()
    coreg_file = os.path.abspath(coreg_filename)
    
    return(coreg_file)

make_coreg_img = Node(name='make_coreg_img',
                      interface=Function(input_names=['epi','anat'],
                                         output_names=['coreg_file'],
                                         function=create_coreg_plot))

In [ ]:
## Nodes for preprocessing

#Slice timing correction based on interleaved acquisition using FSL
slicetime_correct = MapNode(SliceTimer(interleaved=interleave, 
                                       custom_timings=custom_timings,
                                       time_repetition=TR, 
                                       out_file='st_func.nii.gz'),
                            name='slicetime_correct', iterfield=['in_file'])
# Rigid realignment
realign = MapNode(MCFLIRT(out_file='rest_moco.nii.gz'), 
                  name='realign', iterfield=['in_file'])

# compute DVARS and FD
motion_metrics = MapNode(MotionOutliers(out_metric_values='out_mot_metric.txt',
                                        out_metric_plot='plot.png'),
                         name='motion_metrics',iterfield=['in_file'])
motion_metrics.iterables(metric,['fd','dvars'])

# Registration- using bbregister
coreg = MapNode(BBRegister(contrast_type='bold', subjects_dir=fs_dir), name='coreg1', iterfield=['source_file'])

In [ ]:
## Preprocessing Workflow
fmripreproc = Workflow(name='fmri_preprocflow')
fmripreproc.connect([(infosource,selectanat,[('subject_id','subject_id')]), 
                     (infosource,selectfunc,[('subject_id','subject_id')]), 
                     (infosource,coreg,[('subject_id','subject_id')]), 
                     
                     (selectfunc,reorientfunc,[('func','in_file')]),
                     (reorientfunc,slicetime_correct,[('out_file','in_file')]),
                     (slicetime_correct,realign,[('slice_time_corrected_file','in_file')]),
                     (realign,coreg,[('out_file','in_file')]),
                     
                     (selectanat,make_coreg_img,[('out_file','anat')]),
                     (coreg,make_coreg_img,[('out_file','epi')]),
                     (make_coreg_img,datasink,[('coreg_file','coreg_image')]),
                   
                     (realign, datasink,[('par_file','motion_parameters')]),
                     (coreg, datasink, [('out_file','registered_func')])
                    ])
fmripreproc.base_dir = workflow_dir
fmripreproc.write_graph(graph2use='flat')
fmripreproc.run('MultiProc', plugin_args={'n_procs': proc_cores})

## Create Nuissance Regressors
These nodes and workflow creates both the subject specific and general nuissance regressors needed for preprocessing the rest data per the process developed by David Montez. 

In [ ]:
# Data grabber- select fMRI
func_template = {'func':output_dir + '/registered_func/%s/*/realigned_func.nii.gz'}
selectfunc = Node(DataGrabber(sort_filelist = True,
                              template = output_dir + '/registered_func/%s/*/realigned_func.nii.gz',
                              field_template = func_template,
                              base_directory = output_dir,
                              infields=['subject_id'], 
                              template_args = {'func':[['subject_id']]}), name='selectfunc')

# select motion params
mot_template={'motion':output_dir + '/motion_parameters/%s/*/rest_moco.nii.gz.par'}
selectmotion = Node(DataGrabber(sort_filelist = True,
                                template = output_dir + '/motion_parameters/%s/*/rest_moco.nii.gz.par',
                                field_template = mot_template,
                                base_directory = output_dir,
                                infields = ['subject_id'], 
                                template_args = {'motion':[['subject_id']]}), name='selectmotion')     

In [ ]:
def mask_blur_func(mask, in_file):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from os.path import abspath
    from numpy import median, where
    from nipype.interfaces.fsl import ApplyMask
    from glob import glob
    from subprocess import check_call

    applymask = ApplyMask()
    applymask.inputs.mask_file = mask
    applymask.inputs.in_file = in_file
    applymask.inputs.out_file = 'masked_file.nii.gz'
    applymask.inputs.nan2zeros = True
    applymask.run()

    masked_file = abspath('masked_file.nii.gz')
    
    
    if 'nonbrain' in mask:
        check_call(['gunzip',masked_file])
        
        from nipype.interfaces.spm import Smooth
        smooth = Smooth()
        smooth.inputs.in_files = 'masked_file.nii'
        smooth.inputs.fwhm = [22,4,4]
        smooth.inputs.out_prefix = 'blurred_'
        smooth.run()
        check_call(['gzip','blurred_masked_file.nii'])
        
    else:
        from nipype.interfaces.fsl import Smooth
        smooth = Smooth()
        smooth.inputs.in_file = masked_file
        smooth.inputs.smoothed_file = 'blurred_masked_file.nii.gz'
        smooth.inputs.fwhm = 4
        smooth.run()

    blurred_masked_file = abspath('blurred_masked_file.nii.gz')

    return(blurred_masked_file)

def leadlagmatrix(motion_file):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from os.path import abspath
    import numpy as np

    motion_params = np.loadtxt(motion_file, dtype=float)
    trs = motion_params.shape[0]
    params = motion_params.shape[1]
    derivatives = np.gradient(motion_params, axis=0)
    leadlagderivs = np.zeros((trs,params*6))
    derivativessq = derivatives**2
    leadlagderivssq = np.zeros((trs,params*6))

    for i in range(0,params):
        for j in range(0,6):
            leadlagderivs[:,j+params*i] =  np.roll(derivatives[:,i],shift=j, axis=0)
            leadlagderivs[:j,j+params*i] = 0

    for i in range(0,params):
        for j in range(0,6):
            leadlagderivssq[:,j+params*i] =  np.roll(derivativessq[:,i],shift=j, axis=0)
            leadlagderivssq[:j,j+params*i] = 0

    np.savetxt('derivsleadlag.txt', leadlagderivs)
    np.savetxt('derivssqleadlag.txt', leadlagderivssq)

    leadlagderivsmot = abspath('derivsleadlag.txt')
    leadlagderivssqmot = abspath('derivssqleadlag.txt')
    
    return(leadlagderivsmot, leadlagderivssqmot)

In [ ]:
# get scanner noise
session_noise = MapNode(Function(input_names=['mask','in_file'], 
                                 output_names=['blurred_masked_file'],
                                 function=mask_blur_func), name='session_noise', iterfield=['in_file'])
session_noise.inputs.mask=template_nonbrain

# get noise associated with WM and CSF
wmcsf_noise = MapNode(Function(input_names=['mask','in_file'], 
                               output_names=['blurred_masked_file'],
                               function=mask_blur_func), name='wmcsf_noise', iterfield=['in_file'])
wmcsf_noise.inputs.mask=template_nongm

# extract components from session nifti
comp_session_noise = MapNode(CompCor(repetition_time=TR,
                                  num_components=9,
                                  components_file='components.txt',
                                  mask_files=template_mask), 
                          name='comp_session_noise', iterfield=['realigned_file'])

# extract components from WM-CSF nifti 
comp_wmcsf_noise = MapNode(CompCor(repetition_time=TR, 
                                   num_components=9,
                                   components_file='components.txt',
                                   mask_files=template_mask), 
                           name='comp_wmcsf_noise', iterfield=['realigned_file'])

# prepare leadlag motion and derivatives
prep_motion = MapNode(Function(input_names=['motion_file'], 
                               output_names=['leadlagderivsmot','leadlagderivssqmot'],
                               function=leadlagmatrix), 
                      name='prep_motion', iterfield=['motion_file'])

In [ ]:
create_noise_flow = Workflow(name='create_noise_flow')
create_noise_flow.connect([(infosource,selectfunc,[('subject_id','subject_id')]),
                           (selectfunc, wmcsf_noise, [('func','in_file')]),
                           (selectfunc, session_noise, [('func','in_file')]),
                           (wmcsf_noise, comp_wmcsf_noise, [('blurred_masked_file','realigned_file')]),
                           (session_noise, comp_session_noise, [('blurred_masked_file','realigned_file')]),
                           (wmcsf_noise, datasink, [('blurred_masked_file','wmcsf_noise_file')]),
                           (session_noise, datasink, [('blurred_masked_file','session_noise_file')]),
                           (comp_wmcsf_noise, datasink, [('components_file','subject_wmcsf_comp_noise')]),
                           (comp_session_noise, datasink, [('components_file','subject_session_comp_noise')]),
                           
                           (infosource,selectmotion,[('subject_id','subject_id')]),
                           (selectmotion, prep_motion, [('motion','motion_file')]),
                           (prep_motion, datasink, [('leadlagderivsmot','leadlagderivsmotion'),
                                                    ('leadlagderivssqmot','leadlagderivs_squaremotion')])
                          ])
create_noise_flow.base_dir = workflow_dir
create_noise_flow.write_graph(graph2use='flat')
create_noise_flow.run('MultiProc', plugin_args={'n_procs': 4})

### Create mean volume and extract the first 9 principle components

In [ ]:
from glob import glob
from nipype import config, logging
config.enable_debug_mode()
logging.update_logging(config)
from os import mkdir, remove
from os.path import abspath
from nibabel import load, save, Nifti1Image
import numpy as np
from subprocess import check_call

mkdir(output_dir + '/mean_func')
mf_output_dir = output_dir + '/mean_func'

in_files = glob(output_dir + '/registered_func/*/*/realigned_func.nii.gz')
    
image = load(in_files[0])
data = image.get_data()
data = np.expand_dims(data,4)
maxt = data.shape[3]

for a in range(1,len(in_files)):
    tempimg = load(in_files[a])
    tempdata = tempimg.get_data()
    if tempdata.shape[3] > maxt:
        maxt=tempdata.shape[3]

files_to_avg = []        

for a in range(0,len(in_files)):
    tempimg = load(in_files[a])
    tempdata = tempimg.get_data()
    if tempdata.shape[3] < maxt:
        padn=data.shape[3]-tempdata.shape[3]
        tempdata=np.pad(tempdata,pad_width=((0,0),(0,0),(0,0),(0,padn)),
                        mode='constant',constant_values=0)
        new_img = Nifti1Image(tempdata, affine=tempimg.affine, header=tempimg.header)
        save(new_img, 'realigned_func_pad_{0}.nii.gz'.format(a))
        files_to_avg.append(abspath('realigned_func_pad_{0}.nii.gz'.format(a)))
    elif tempdata.shape[3] == maxt:
        files_to_avg.append(in_files[a])
        
mean_file=mf_output_dir+'/mean_funcs.nii.gz'

check_call(['3dMean','-prefix',mean_file,'-non_zero'] +files_to_avg)

In [ ]:
# extract components from averaged time series
comp_average_noise = CompCor()
comp_average_noise.inputs.realigned_file=mean_file
comp_average_noise.inputs.repetition_time=TR
comp_average_noise.inputs.num_components=9
comp_average_noise.inputs.components_file=mf_output_dir +'/components.txt' 
comp_average_noise.inputs.mask_files=template_mask
comp_average_noise.run()

In [ ]:
import matplotlib.pyplot as plt
from pandas import read_csv

components = read_csv('/Users/catcamacho/Dropbox/temp/babies_20191010/mean_func/components.txt', 
                      index_col=None, delimiter='\t')

for comp in components.columns:
    plt.figure()
    plt.plot(components[comp])
    plt.show()
    plt.close()

## Movement Noise Evaluation Workflow

The nodes and workflow below (eval_noise_flow) is designed to take the nuissance regressors created in the previous section (create_noise_flow) and perform voxel-specific assessment of noise.  This is accomplished through the following steps:
1. Create unique design matrix for each 3D voxel
2. Perform a GLM for that voxel
3. Project results back into 3D space

In [ ]:
sub_files_template={'motion': output_dir + '/motion_parameters/{subject_id}/_realign{runnum}/rest_moco.nii.gz.par', 
                    'leadlagderivsmotion': output_dir + '/leadlagderivsmotion/{subject_id}/_prep_motion{runnum}/derivsleadlag.txt', 
                    'leadlagderivs_squaremotion': output_dir + '/leadlagderivs_squaremotion/{subject_id}/_prep_motion{runnum}/derivssqleadlag.txt', 
                    'func': output_dir + '/registered_func/{subject_id}/_xfmFUNC{runnum}/realigned_func.nii.gz'}
select_sub_files=Node(SelectFiles(sub_files_template),name='select_sub_files')
select_sub_files.iterables=('runnum',['0','1'])

In [ ]:
def org_shared_noise(motion, leadlagderivsmotion, leadlagderivs_squaremotion):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from numpy import loadtxt, concatenate
    from pandas import DataFrame
    from os.path import abspath
    
    noise_list = []
    for file in [motion, leadlagderivsmotion, leadlagderivs_squaremotion]:
        mo = loadtxt(file, dtype=float, comments=None)
        length_of_file = mo.shape[0]
        noise_list.append(mo)

    shared_noise_data = concatenate(noise_list,axis=1)
    
    col_names = ['noise_{0}'.format(a) for a in range(0,shared_noise_data.shape[1])] 
    
    shared_noise = DataFrame(shared_noise_data, columns=col_names)
    shared_noise.to_csv('shared_noise.csv')
    shared_noise_file = abspath('shared_noise.csv')
    return(shared_noise_file)

def voxelwise_p_eta(func,shared_noise_file,mean_func,mask):
    from os.path import abspath
    from numpy import zeros, ones
    from numpy.linalg import lstsq
    from pandas import read_csv, Series
    from nilearn.masking import apply_mask, unmask

    # import data into an array that is timepoints (rows) by voxel number (columns)
    shared_noise = read_csv(shared_noise_file, index_col=0)
    func_data = apply_mask(func, mask)
    mean_func_data = apply_mask(mean_func, mask)
    mean_func_data = mean_func_data[:func_data.shape[0],:]
    coefficients = zeros((shared_noise.shape[1]+2,func_data.shape[1]))
    num_regress_groups = (shared_noise.shape[0]-6)/3
    p_eta_data = zeros((num_regress_groups,func_data.shape[1]))
    
    # perform voxel-wise matrix inversion
    for x in range(0,func_data.shape[1]):
        shared_noise['mean_signal'] = mean_func_data[:,x]
        shared_noise['constant'] = 1
        noise_mat = shared_noise.to_numpy()
        y = func_data[:,x]
        solution = lstsq(noise_mat,y)
        coefficients[:,x] = solution[0]
        r2 = 1 - solution[1] / (shared_noise.shape[0] * y.var())
        for i in range(0,num_regress_groups):
            temp = shared_noise.drop(['noise_{0}'.format(3*i+6),'noise_{0}'.format(3*i+7),'noise_{0}'.format(3*i+8)],axis=1)
            temp_mat = temp.to_numpy()
            temp_sol = lstsq(temp_mat,y)
            temp_r2 = 1 - solution[1] / (temp_mat.shape[0] * y.var())
            p_eta_data[i,x] = r2-temp_r2  
            
    coeff_image = unmask(coefficients, mask)
    coeff_image.to_filename('weights.nii.gz')
    p_eta_image = unmask(p_eta_data, mask)
    p_eta_image.to_filename('p_eta.nii.gz')
    sample_design_df = shared_noise.to_csv('last_noise_mat.csv')

    weights = abspath('weights.nii.gz')
    p_eta = abspath('p_eta.nii.gz')
    sample_design_df = abspath('last_noise_mat.csv')

    return(weights,p_eta, sample_design_df)

In [ ]:
compile_noise_mat = Node(Function(input_names=['motion', 'leadlagderivsmotion', 'leadlagderivs_squaremotion'],
                                  output_names=['shared_noise_file'],
                                  function=org_shared_noise), 
                         name='compile_noise_mat')
eval_noise_func = Node(Function(input_names=['func','shared_noise_file','mean_func','mask'], 
                                output_names=['weights','p_eta','sample_design_df'],
                                function=voxelwise_p_eta),
                       name='eval_noise_func')
eval_noise_func.inputs.mask = template_maskD1
eval_noise_func.inputs.mean_func = output_dir + '/mean_func/mean_funcs.nii.gz'

In [ ]:
eval_noise_flow = Workflow(name='eval_noise_flow')
eval_noise_flow.connect([(infosource, select_sub_files,[('subject_id','subject_id')]),
                         (select_sub_files, eval_noise_func, [('func','func')]),
                         (select_sub_files, compile_noise_mat, [('motion','motion'),
                                                                ('leadlagderivsmotion','leadlagderivsmotion'), 
                                                                ('leadlagderivs_squaremotion','leadlagderivs_squaremotion')]),
                         (compile_noise_mat, eval_noise_func, [('shared_noise_file','shared_noise_file')]),
                         
                         (eval_noise_func,datasink,[('weights','denoising_weights'),
                                                    ('sample_design_df','denoise_sample_design_df'),
                                                    ('p_eta','p_eta')])
                        ])
eval_noise_flow.base_dir = workflow_dir
eval_noise_flow.write_graph(graph2use='flat')
eval_noise_flow.run('MultiProc', plugin_args={'n_procs': 3})

### Assess which motion parameters are associated with systematic noise

In [ ]:
# create mean of the parameter estimates for the noise variables

from subprocess import check_call
from glob import glob
files_to_avg = glob(output_dir + '/denoising_weights/*/*/weights.nii.gz')
check_call(['3dMean','-prefix',output_dir + '/mean_weights/mean_noise_estimates.nii.gz','-non_zero'] +files_to_avg)

files_to_avg = glob(output_dir + '/p_eta/*/*/p_eta.nii.gz')
check_call(['3dMean','-prefix',output_dir + '/mean_p_eta/p_eta_mean.nii.gz','-non_zero'] +files_to_avg)

In [ ]:
from nilearn.masking import apply_mask
import matplotlib.pyplot as plt
from numpy import mean

p_eta_data = apply_mask(output_dir + '/mean_p_eta/p_eta_mean.nii.gz', template_gmmask)
mean_p_etas = mean(p_eta_data, axis=1)
indices_to_drop = sorted(range(len(mean_p_etas)), key=lambda i: mean_p_etas[i])[:6]
print(indices_to_drop)

noise_to_drop = []
for i in indices_to_drop:
    noise_to_drop.append('noise_{0}'.format(3*i+6))
    noise_to_drop.append('noise_{0}'.format(3*i+7))
    noise_to_drop.append('noise_{0}'.format(3*i+8))
plt.plot(mean_p_etas)

## Final Denoising Workflow

In [ ]:
sub_files_template={'func': output_dir + '/registered_func/{subject_id}/_xfmFUNC{runnum}/realigned_func.nii.gz',
                    'shared_noise_file': output_dir + '/shared_noise_file/{subject_id}/_runnum{runnum}/shared_noise.csv'}
select_sub_files=Node(SelectFiles(sub_files_template),name='select_sub_files')
select_sub_files.iterables=('runnum',['0','1'])

In [ ]:
def voxelwise_glm(func,shared_noise_file,mean_func,mask,noise_to_drop):
    from os.path import abspath
    from numpy import zeros, dot, transpose, sum
    from numpy.linalg import pinv
    from pandas import read_csv, Series
    from nilearn.masking import apply_mask, unmask

    # import data into an array that is timepoints (rows) by voxel number (columns)
    shared_noise = read_csv(shared_noise_file, index_col=0)
    shared_noise.drop(labels=noise_to_drop, inplace=True, axis=1)
    func_data = apply_mask(func, mask)
    mean_func_data = apply_mask(mean_func, mask)
    mean_func_data = mean_func_data[:func_data.shape[0],:]
    coefficients = zeros((shared_noise.shape[1]+2,func_data.shape[1]))
    resid_data = zeros(func_data.shape)

    # perform voxel-wise matrix inversion
    for x in range(0,func_data.shape[1]):
        shared_noise['mean_signal'] = mean_func_data[:,x]
        shared_noise['constant'] = 1
        noise_mat = shared_noise.to_numpy()
        y = func_data[:,x]
        inv_mat = pinv(noise_mat)
        coefficients[:,x] = dot(inv_mat,y)
        yhat=sum(transpose(coefficients[:,x])*noise_mat,axis=1)
        resid_data[:,x] = y - transpose(yhat)

    resid_image = unmask(resid_data, mask)
    resid_image.to_filename('residuals.nii.gz')

    coeff_image = unmask(coefficients, mask)
    coeff_image.to_filename('weights.nii.gz')
    sample_design_df = shared_noise.to_csv('last_noise_mat.csv')

    weights = abspath('weights.nii.gz')
    sample_design_df = abspath('last_noise_mat.csv')
    residuals = abspath('residuals.nii.gz')

    return(weights,sample_design_df, residuals)

def convertafni(in_file):
    from nipype.interfaces.afni.utils import AFNItoNIFTI
    from os.path import abspath
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    cvt = AFNItoNIFTI()
    cvt.inputs.in_file = in_file
    cvt.inputs.out_file = 'func_filtered.nii.gz'
    cvt.run()
    
    out_file = abspath('func_filtered.nii.gz')
    return(out_file)

def drop_high_motion_trs(in_file, mask, motion):
    from os.path import abspath
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nilearn.masking import apply_mask, unmask
    import numpy as np

    motion_params = np.loadtxt(motion)
    motion_derivs = np.gradient(motion_params, axis=0)
    fd = np.sum(np.absolute(motion_derivs),axis=1)

    func_data = apply_mask(in_file,mask)
    lomo_func = func_data[fd<0.5]
    lomo_image = unmask(lomo_func,mask)
    lomo_image.to_file('lomo_func.nii.gz')
    out_file = abspath('lomo_func.nii.gz')
    
    return(out_file)

In [ ]:
# band pass filtering- all rates are in Hz (1/TR or samples/second)
denoise_func = Node(Function(input_names=['func','shared_noise_file','mean_func','mask','noise_to_drop'], 
                             output_names=['weights','sample_design_df','residuals'],
                             function=voxelwise_glm),
                       name='denoise_func')
denoise_func.inputs.mask = template_maskD1
denoise_func.inputs.mean_func = output_dir + '/mean_func/mean_funcs.nii.gz'
denoise_func.inputs.noise_to_drop = noise_to_drop

bandpass = Node(Bandpass(highpass=highpass_freq,
                         lowpass=lowpass_freq), 
                name='bandpass')

afni_convert = Node(Function(input_names=['in_file'],
                             output_names=['out_file'],
                             function=convertafni), 
                    name='afni_convert')

drop_himo = Node(Function(input_names=['in_file','mask','motion'], 
                          output_names=['out_file'], 
                          function=drop_high_motion_trs), 
                 name=drop_himo)

merge = JoinNode(Merge(dimension='t'), joinsource='select_sub_files', 
                 joinfield=['runnum'], name='merge')

In [ ]:
denoise_flow = Workflow(name='denoise_flow')
denoise_flow.connect([(infosource, select_sub_files,[('subject_id','subject_id')]),
                      (select_sub_files, denoise_func, [('func','func'),
                                                        ('shared_noise_file','shared_noise_file')]),
                      (denoise_func,bandpass,[('residuals','in_file')]),
                      (bandpass,afni_convert,[('out_file','in_file')]),
                      (afni_convert, drop_himo, [('out_file','in_file')]),
                      (drop_himo,merge,[('out_file','in_files')]),
                      
                      (merge,datasink,[('out_file','fully_processed_func')]),
                      (denoise_func,datasink,[('weights','denoising_weights'),
                                              ('sample_design_df','denoise_sample_design_df')])
                     ])
denoise_flow.base_dir = workflow_dir
denoise_flow.write_graph(graph2use='flat')
denoise_flow.run('MultiProc', plugin_args={'n_procs': 3})